In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

#Ruta raiz
PATH = "/content/drive/My Drive/Challe/"
#datos de entrada 
INPATH = PATH + "data/"
#datos de Salida
OUTPATH = PATH + "tg_img/"
#datos de Salida
ChkPATH = PATH + "Checkpoints"

imgurls=!ls -1 "{INPATH}"

n=700
train_n = round(n*.8)

#Aleatorizar 
randurls = np.copy(imgurls)

#np.random.seed(23) #esto no cuenta para hacerlo realmente aleatorio se usa una seed para que queden igual al tutoriall
np.random.shuffle(randurls)

#Particionar train/test dataset
tr_urls = randurls[:train_n]
ts_urls= randurls[train_n:n]
print(len(imgurls),len(tr_urls),len(ts_urls))

952 560 140


In [0]:
import random
#Cargar Imagenes
IMG_WIDTH =256
IMG_HEIGHT =256
#rezize
def resize(inimg,tgimg, height,width):
  inimg = tf.image.resize(inimg,[height,width])
  tgimg = tf.image.resize(tgimg,[height,width])
  
  return inimg,tgimg

#Normalizar de -1 a 1 la imagen
def normalize(inimg,tgimg):
  inimg = (inimg/127.5)-1
  tgimg = (tgimg/127.5)-1
  
  return inimg,tgimg

#Aumentar datos crop+flip
def random_jitter(inimg, tgimg):
  inimg,tgimg = resize(inimg,tgimg,286,286) 
  #apilar
  stacked_image = tf.stack([inimg,tgimg], axis=0)
  #recortar
  cropped_image = tf.image.random_crop(stacked_image, size=[2,IMG_HEIGHT,IMG_WIDTH,3])

  inimg,tgimg=cropped_image[0],cropped_image[1]

  if random() > .5:
    inimg= tf.image.flip_left_right(inimg)
    tgimg= tf.image.flip_left_right(tgimg)

  return inimg,tgimg

 #Cargar imagenes 
def load_image(filename, augment=True):
  inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(INPATH+filename),channels=3),tf.float32)[...,:3]
  tgimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(OUTPATH+filename)),tf.float32)[...,:3]
  inimg, tgimg =resize(inimg,tgimg,IMG_HEIGHT,IMG_WIDTH)
  if augment:
    #inimg,tgimg = random_jitter(inimg,tgimg)
    inimg,tgimg = normalize(inimg,tgimg)
  return inimg,tgimg

def load_train_img(filename):
  return load_image(filename)

def load_test_img(filename):
  return load_image(filename,False)
#avf=load_train_img(randurls[1])
#inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(INPATH+randurls[0]),channels=3),tf.float32)[...,:3]
#inimg = tf.cast(tf.io.decode_gif(tf.io.read_file(INPATH+randurls[10])),tf.float32)[...,:3]
#tgimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(OUTPATH+randurls[10])),tf.float32)[...,:3]
#print (inimg.shape)
#plt.imshow(((load_train_img(randurls[0])[0])+1)/2)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
train_dataset = train_dataset.map(load_train_img,num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(1)

test_dataset = tf.data.Dataset.from_tensor_slices(ts_urls)
test_dataset = test_dataset.map(load_test_img,num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(1)


In [0]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
def downsample(filters, apply_batchnorm=True):
  result = Sequential()

  initializer = tf.random_normal_initializer(0,0.02)

  #agregar capa convolucional
  result.add(Conv2D(filters,
             kernel_initializer=initializer,
             kernel_size=4,
             strides=2,
             padding="same",
             use_bias=not apply_batchnorm))
  #capa de BatchNorm
  if apply_batchnorm:
    result.add(BatchNormalization())

  #capa de activacion leaky_ReLU
  result.add(LeakyReLU())

  return result
  #downsample(64)

In [0]:
def upsample(filters, apply_dropout=False):
  result = Sequential()

  initializer = tf.random_normal_initializer(0,0.02)

  #agregar capa convolucional
  result.add(Conv2DTranspose(filters,
                            kernel_initializer=initializer,
                            kernel_size=4,
                            strides=2,
                            padding="same",
                            use_bias=False))
  #batch norm
  result.add(BatchNormalization())
  #capa de Dropout
  if apply_dropout:
    result.add(Dropout(.5))
  #capa de activacion ReLU
  result.add(ReLU())

  return result
#upsample(64)

In [0]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[None,None,3])
  down_stack = [
                downsample(64, apply_batchnorm=False),  #(bs,128,128,64)
                downsample(128),                        #(bs,64,64,128)
                downsample(256),                        #(bs,32,32,256)
                downsample(512),                        #(bs,16,16,512)
                downsample(512),                        #(bs,8,8,512)
                downsample(512),                        #(bs,4,4,512)
                downsample(512),                        #(bs,2,2,512)
                downsample(512),                         #(bs,1,1,512)
  ]
  up_stack = [
                upsample(512, apply_dropout=True),      #(bs,2,2,   1024)
                upsample(512, apply_dropout=True),      #(bs,4,4,   1024)
                upsample(512, apply_dropout=True),      #(bs,8,8,   1024)
                upsample(512),                          #(bs,16,16, 1024)
                upsample(256),                          #(bs,32,32,  512)
                upsample(128),                          #(bs,64,64,  256)
                upsample(64),                            #(bs,128,128,128)
  ]
  initializer = tf.random_normal_initializer(0,0.02)
  last = Conv2DTranspose(filters=3,
                         kernel_size=4,
                         strides = 2,
                         padding="same",
                         kernel_initializer = initializer,
                         activation = "tanh")
  x = inputs
  s = []
  concat = Concatenate()
  for down in down_stack:
    x=down(x)
    s.append(x)  

  s=reversed(s[:-1])
  
  for up,sk in zip(up_stack,s):
    x=up(x)
    x=concat([x,sk])
  
  last=last(x)
  return Model(inputs=inputs,outputs=last)


In [0]:
def Discriminator():
  ini = Input(shape=[None, None, 3],name="input_img")
  gen = Input(shape=[None, None, 3],name="gener_img")

  con=concatenate([ini,gen])

  initializer = tf.random_normal_initializer(0,.02)

  down1 = downsample(64,apply_batchnorm=False)(con)
  down2 = downsample(128)(down1)
  down3 = downsample(256)(down2)
  down4 = downsample(512)(down3)

  last = tf.keras.layers.Conv2D(filters=1,
                                kernel_size=4,
                                strides=1,
                                kernel_initializer=initializer,
                                padding="same")(down4)

  return tf.keras.Model(inputs=[ini,gen], outputs=last)


In [0]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)  

In [0]:
def discriminator_loss(disc_real_output, disc_generated_output):

  #Diferencias entre los true por ser real y el detectado por discriminador.
  real_loss = loss_object(tf.ones_like(disc_real_output),disc_real_output)
  
  #Diferencias entre los false por ser generado y el detectado por discriminador.
  generated_loss = loss_object(tf.zeros_like(disc_generated_output),disc_generated_output)
  
  total_disc_loss = real_loss + generated_loss
  
  return total_disc_loss

In [0]:
LAMBDA = 100

def generator_loss(disc_generated_output, gen_output, target):
 
  gan_loss = loss_object(tf.ones_like(disc_generated_output),disc_generated_output)

  #mean absolute_error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  
  total_gen_loss = gan_loss + (LAMBDA * l1_loss)
  return total_gen_loss

In [13]:
import os

generator = Generator()
discriminator = Discriminator()

generator_optimizer      = tf.keras.optimizers.Adam(2e-4,beta_1=.5)
discriminator_optimizer  = tf.keras.optimizers.Adam(2e-4,beta_1=.5)

checkpoint_prefix = os.path.join(ChkPATH,"ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
#checkpoint.restore('/content/drive/My Drive/IANB/Checkpoints/ckpt-9.index')
checkpoint.restore(tf.train.latest_checkpoint(ChkPATH))

In [0]:
def generate_images(model,test_input,tar,save_filename=False, display_imgs=True):
  
  prediction = model(test_input,training=True)

  if save_filename:
    tf.keras.preprocessing.image.save_img(PATH + 'output/' + save_filename + '.jpg', prediction[0,...])
  
  plt.figure(figsize=(10,10))

  display_list = [test_input[0],tar[0],prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  if display_imgs:
    for i in range (3):
      plt.subplot(1,3,i+1)
      plt.title(title[i])
      #toamndo los valores en el dominio [0,1] y plot 
      plt.imshow(display_list[i]*.5 +.5)
      plt.axis('off')
  plt.show()

In [0]:
@tf.function()
def train_step(input_image,target):

  with tf.GradientTape() as gen_tape, tf.GradientTape() as discr_tape:
 
    output_img = generator(input_image,training=True)

    output_gen_discr = discriminator([output_img,input_image],training=True)

    output_trg_discr = discriminator([target,input_image],training=True)

    discr_loss=discriminator_loss(output_trg_discr,output_gen_discr)

    gen_loss = generator_loss(output_gen_discr,output_img,target)

    generator_grads = gen_tape.gradient(gen_loss,generator.trainable_variables)

    discriminator_grads = discr_tape.gradient(discr_loss,discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_grads,generator.trainable_variables))

    discriminator_optimizer.apply_gradients(zip(discriminator_grads,discriminator.trainable_variables))

In [0]:
from IPython.display import clear_output

def train(dataset,epochs):
  for epoch in range(epochs):

    imgi = 0
    for input_image,target in dataset:
      print('epoch' + str(epoch) + 'train: ' + str(imgi) + '/' + str(len(tr_urls)))
      imgi+=1
      train_step(input_image,target)
      clear_output(wait=True)
    
    imgi = 0
    for inp,tar in test_dataset.take(5):
      generate_images(generator,inp,tar,str(imgi) + '_' + str(epoch),display_imgs=False)
      imgi+=1
    #Guardar ckpt cada 50 epochs
    if((epoch+1)%25) == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)



In [0]:
train(train_dataset,600)

epoch5train: 385/560
